1. Implement the CKY algorithm and test it with your converted L1 grammar.


In [ ]:
def cky_parse(grammar, sentence):
  """
  Implements the CKY parsing algorithm.

  Args:
    grammar: A dictionary representing the CNF grammar.
    sentence: A list of words representing the input sentence.

  Returns:
    A list of parse trees, where each parse tree is a tuple of the form (non-terminal, left index, right index, children).
  """

  # Initialize the chart.
  chart = [[[] for _ in range(len(sentence) + 1)] for _ in range(len(sentence) + 1)]

  # Fill in the base cases.
  for i, word in enumerate(sentence):
    for non_terminal, productions in grammar.items():
      if word in productions:
        chart[i][i + 1].append((non_terminal, i, i + 1, []))

  # Iterate over the chart, filling in the cells.
  for length in range(2, len(sentence) + 1):
    for start in range(len(sentence) - length + 1):
      end = start + length
      for split in range(start + 1, end):
        left_children = chart[start][split]
        right_children = chart[split][end]

        for left_non_terminal, left_start, left_end, _ in left_children:
          for right_non_terminal, right_start, right_end, _ in right_children:
            for non_terminal, productions in grammar.items():
              if f"{left_non_terminal} {right_non_terminal}" in productions:
                chart[start][end].append((non_terminal, left_start, right_end, [(left_non_terminal, left_start, left_end, []), (right_non_terminal, right_start, right_end, [])]))

  # Return the parse trees for the start symbol.
  return chart[0][len(sentence)]

# Define the CNF grammar (converted L1 grammar).
cnf_grammar = {
    "S": ["NP VP", "Aux NP VP", "VP", "V NP", "VP PP", "V", "NP", "Det N", "Det", "N", "V", "P"],
    "NP": ["P", "Det N", "Det N PP", "PRP"],
    "VP": ["V", "V NP", "V NP PP", "V PP", "VP"],
    "PP": ["P NP"],
    "Det": ["the", "a"],
    "N": ["man", "park", "dog"],
    "PRP": ["I"],
    "P": ["in", "with", "on", "by", "to"],
    "V": ["saw", "ate", "walked", "ran"],
    "Aux": ["did", "does", "do"]
}

# Test the CKY algorithm with the converted L1 grammar.
sentence = "the man saw the dog".split()
parse_trees = cky_parse(cnf_grammar, sentence)

for tree in parse_trees:
  print(tree)


('S', 0, 5, [('NP', 0, 2, []), ('VP', 2, 5, [])])


2. write the CKY algorithm so that it can accept grammars that contain unit productions.


In [ ]:
def cky_parse_with_unit_productions(grammar, sentence):
  """
  Implements the CKY parsing algorithm, allowing for grammars that contain unit productions.

  Args:
    grammar: A dictionary representing the CNF grammar.
    sentence: A list of words representing the input sentence.

  Returns:
    A list of parse trees, where each parse tree is a tuple of the form (non-terminal, left index, right index, children).
  """

  # Initialize the chart.
  chart = [[[] for _ in range(len(sentence) + 1)] for _ in range(len(sentence) + 1)]

  # Fill in the base cases.
  for i, word in enumerate(sentence):
    for non_terminal, productions in grammar.items():
      if word in productions:
        chart[i][i + 1].append((non_terminal, i, i + 1, []))

  # Iterate over the chart, filling in the cells.
  for length in range(2, len(sentence) + 1):
    for start in range(len(sentence) - length + 1):
      end = start + length
      for split in range(start + 1, end):
        left_children = chart[start][split]
        right_children = chart[split][end]

        for left_non_terminal, left_start, left_end, _ in left_children:
          for right_non_terminal, right_start, right_end, _ in right_children:
            for non_terminal, productions in grammar.items():
              if f"{left_non_terminal} {right_non_terminal}" in productions:
                chart[start][end].append((non_terminal, left_start, right_end, [(left_non_terminal, left_start, left_end, []), (right_non_terminal, right_start, right_end, [])]))
              elif f"{left_non_terminal} -> {right_non_terminal}" in productions:
                chart[start][end].append((non_terminal, left_start, right_end, [(left_non_terminal, left_start, right_end, [])]))
              elif f"{right_non_terminal} -> {left_non_terminal}" in productions:
                chart[start][end].append((non_terminal, left_start, right_end, [(right_non_terminal, left_start, right_end, [])]))

  # Return the parse trees for the start symbol.
  return chart[0][len(sentence)]

# Test the CKY algorithm with the provided grammar and sentence.
grammar = {
    "S": ["NP VP", "Aux NP VP", "VP", "V NP", "VP PP", "V", "NP", "Det N", "Det", "N", "V", "P"],
    "NP": ["P", "Det N", "Det N PP", "PRP"],
    "VP": ["V", "V NP", "V NP PP", "V PP", "VP"],
    "PP": ["P NP"],
    "Det": ["the", "a"],
    "N": ["man", "park", "dog"],
    "PRP": ["I"],
    "P": ["in", "with", "on", "by", "to"],
    "V": ["saw", "ate", "walked", "ran"],
    "Aux": ["did", "does", "do"],
    "Aux -> V": ["does", "do"]  # Example of unit production
}

sentence = "the man saw the dog".split()
parse_trees = cky_parse_with_unit_productions(grammar, sentence)

for tree in parse_trees:
  print(tree)


('S', 0, 5, [('NP', 0, 2, []), ('VP', 2, 5, [])])


Implement an ELIZA-like program, using substitutions. You might want to choose a different domain than a Rogerian psychologist, although keep in mind that you would need a domain in which your program can legitimately engage in a lot of simple repetition.

In [ ]:

import re

# Define a dictionary of substitutions.
substitutions = {
    "I": "you",
    "me": "you",
    "my": "your",
    "myself": "yourself",
    "you": "I",
    "your": "my",
    "yours": "mine",
    "yourself": "myself",
    "am": "are",
    "are": "am",
    "was": "were",
    "were": "was",
    "have": "have",
    "has": "have",
    "had": "had",
    "do": "do",
    "does": "do",
    "did": "did",
    "will": "will",
    "would": "would",
    "can": "can",
    "could": "could",
    "should": "should",
    "ought": "ought",
    "must": "must",
    "need": "need",
    "feel": "feel",
    "think": "think",
    "believe": "believe",
    "know": "know",
    "understand": "understand",
    "want": "want",
    "desire": "desire",
    "hope": "hope",
    "wish": "wish",
    "dream": "dream",
    "fear": "fear",
    "worry": "worry",
    "doubt": "doubt",
    "regret": "regret",
    "love": "love",
    "hate": "hate",
    "like": "like",
    "dislike": "dislike",
    "trust": "trust",
    "distrust": "distrust",
    "respect": "respect",
    "disrespect": "disrespect",
    "admire": "admire",
    "criticize": "criticize",
    "blame": "blame",
    "praise": "praise",
    "thank": "thank",
    "apologize": "apologize",
    "forgive": "forgive",
    "help": "help",
    "hurt": "hurt",
    "harm": "harm",
    "kill": "kill",
    "die": "die"
}

# Define a function to transform a sentence using substitutions.
def transform_sentence(sentence):
    # Split the sentence into words.
    words = sentence.split()

    # Apply the substitutions to the words.
    for i, word in enumerate(words):
        if word in substitutions:
            words[i] = substitutions[word]

    # Join the words back into a sentence.
    return " ".join(words)

# Define a function to generate an ELIZA-like response.
def generate_response(sentence):
    # Transform the sentence using substitutions.
    transformed_sentence = transform_sentence(sentence)

    # Generate a generic response.
    response = "I see. And how do you feel about that?"

    # If the transformed sentence contains a question, generate a question response.
    if "?" in transformed_sentence:
        response = "Why do you ask that?"

    # Return the response.
    return response

# Start the ELIZA-like program.
while True:
    # Get the user's input.
    sentence = input("You: ")

    # Generate a response.
    response = generate_response(sentence)

    # Print the response.
    print("ELIZA:", response)

    # If the user says "quit", exit the program.
    if sentence == "quit":
        break


You: You
ELIZA: I see. And how do you feel about that?
You: Fine
ELIZA: I see. And how do you feel about that?
You: great
ELIZA: I see. And how do you feel about that?
You: quit
ELIZA: I see. And how do you feel about that?


Implement the algorithm to convert arbitrary context-free grammars to CNF


In [ ]:

def convert_to_cnf(grammar):
  """
  Converts an arbitrary context-free grammar to Chomsky Normal Form (CNF).

  Args:
    grammar: A dictionary representing the context-free grammar.

  Returns:
    A dictionary representing the CNF grammar.
  """

  # Initialize the CNF grammar.
  cnf_grammar = {}

  # Convert all rules to binary rules.
  for non_terminal, productions in grammar.items():
    cnf_grammar[non_terminal] = []
    for production in productions:
      if len(production.split()) == 2:
        cnf_grammar[non_terminal].append(production)
      else:
        new_non_terminal = f"{non_terminal}_{len(cnf_grammar) + 1}"
        cnf_grammar[new_non_terminal] = [production]
        cnf_grammar[non_terminal].append(f"{new_non_terminal} {production.split()[1]}")

  # Eliminate unit productions.
  while True:
    changed = False
    for non_terminal, productions in cnf_grammar.items():
      for production in productions:
        if len(production.split()) == 1 and production in cnf_grammar:
          for new_production in cnf_grammar[production]:
            if new_production not in productions:
              cnf_grammar[non_terminal].append(new_production)
              changed = True
    if not changed:
      break

  # Return the CNF grammar.
  return cnf_grammar


Write a program to compute unsmoothed unigrams and bigrams


In [ ]:
import random

# Generate a random paragraph.
paragraph = " ".join([random.choice(["the", "a", "an", "in", "on", "at", "of", "to", "for", "by", "with", "from", "up", "down", "left", "right", "back", "forward", "again", "once", "twice", "three times", "many times", "a few times", "no times", "all the time", "some of the time", "most of the time", "half of the time", "a quarter of the time", "three quarters of the time", "a third of the time", "two thirds of the time", "one fifth of the time", "four fifths of the time", "one sixth of the time", "five sixths of the time", "one seventh of the time", "six sevenths of the time", "one eighth of the time", "seven eighths of the time", "one ninth of the time", "eight ninths of the time", "one tenth of the time", "nine tenths of the time"]) for _ in range(100)])

# Tokenize the paragraph.
tokens = paragraph.split()

# Compute the unigrams.
unigrams = {}
for token in tokens:
  if token not in unigrams:
    unigrams[token] = 0
  unigrams[token] += 1

# Compute the bigrams.
bigrams = {}
for i in range(len(tokens) - 1):
  bigram = (tokens[i], tokens[i + 1])
  if bigram not in bigrams:
    bigrams[bigram] = 0
  bigrams[bigram] += 1

# Print the unigrams and bigrams.
print("Unigrams:")
for token, count in unigrams.items():
  print(f"{token}: {count}")

print("\nBigrams:")
for bigram, count in bigrams.items():
  print(f"{bigram}: {count}")


Unigrams:
on: 4
a: 10
few: 2
times: 9
three: 5
quarters: 3
of: 41
the: 42
time: 41
back: 3
with: 2
half: 2
for: 1
left: 4
by: 7
in: 3
one: 14
seventh: 6
from: 4
quarter: 3
third: 3
two: 4
thirds: 4
four: 3
fifths: 3
again: 2
five: 1
sixths: 1
eight: 1
ninths: 1
some: 3
ninth: 3
seven: 1
eighths: 1
six: 2
sevenths: 2
right: 4
an: 2
twice: 3
eighth: 2
all: 1
no: 2
up: 2
many: 3
forward: 3
sixth: 1
at: 1
once: 1
fifth: 2

Bigrams:
('on', 'a'): 1
('a', 'few'): 2
('few', 'times'): 2
('times', 'three'): 1
('three', 'quarters'): 3
('quarters', 'of'): 3
('of', 'the'): 40
('the', 'time'): 41
('time', 'back'): 2
('back', 'with'): 1
('with', 'a'): 1
('a', 'half'): 1
('half', 'of'): 2
('time', 'with'): 1
('with', 'for'): 1
('for', 'left'): 1
('left', 'by'): 1
('by', 'in'): 2
('in', 'one'): 1
('one', 'seventh'): 6
('seventh', 'of'): 6
('time', 'on'): 2
('on', 'from'): 1
('from', 'by'): 1
('by', 'a'): 1
('a', 'quarter'): 3
('quarter', 'of'): 3
('time', 'a'): 3
('a', 'third'): 3
('third', 'of'): 3
('

Named Entity Recognition

In [ ]:

import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the text to be processed
text = "Barack Obama was born in Honolulu, Hawaii."

# Process the text
doc = nlp(text)

# Extract the named entities
for entity in doc.ents:
  print(f"{entity.text}: {entity.label_}")


Barack Obama: PERSON
Honolulu: GPE
Hawaii: GPE


TF _ IDF


In [ ]:
# Import necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer with desired parameters
vectorizer = TfidfVectorizer(max_features=5, stop_words='english')

# Define the documents
documents = [
    "This is the first document.",
    "This is the second document.",
    "This document is different."
]

# Fit and transform the documents
X = vectorizer.fit_transform(documents)

# Print the TF-IDF vectors
print(X.toarray())

# Access the vocabulary of the vectorizer
vocabulary = vectorizer.vocabulary_

# Print the vocabulary
print(vocabulary)


[[0.         1.         0.        ]
 [0.         0.50854232 0.861037  ]
 [0.861037   0.50854232 0.        ]]
{'document': 1, 'second': 2, 'different': 0}


Use a library like NLTK or spaCy to perform part-of-speech tagging on
a sentence. Print out the words along with their corresponding parts of speech

In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the sentence to be processed
sentence = "The quick brown fox jumps over the lazy dog."

# Process the sentence
doc = nlp(sentence)

# Print the words and their corresponding parts of speech
for token in doc:
  print(f"{token.text}: {token.pos_}")


The: DET
quick: ADJ
brown: ADJ
fox: NOUN
jumps: VERB
over: ADP
the: DET
lazy: ADJ
dog: NOUN
.: PUNCT
